In [1]:
%load_ext autoreload
%autoreload 2

try:    importlib.reload(Jupytils)
except: import Jupytils
import logging as log
import itertools as it
showTopbar('''Influx DB Test drive''')

try: importlib.reload(Jupytils.Charts)
except: from Jupytils.Charts import *

<IPython.core.display.Javascript object>

In [ ]:
### Run the following commands to create database
#!influx
#CREATE DATABASE mydb

In [1]:
from influxdb import InfluxDBClient
client = InfluxDBClient(host='localhost', port=8086, username=None,password=None,ssl=False, database='mydb')

ImportError: No module named 'influxdb'

In [7]:
#client.create_database('mydb')
client.get_list_database()

[{'name': '_internal'}, {'name': 'mydb'}]

In [27]:
rows= [ 
    { "time": "2018-03-28T8:01:00Z", "measurement": "device1", "tags": { "user": "Carol", "Id": "1" }, "fields": {"val": 127}},
    { "time": "2018-03-28T8:04:00Z", "measurement": "device1", "tags": { "user": "Carol", "Id": "1" }, "fields": {"val": 128}},
]
client.write_points(rows)


True

In [48]:
a=Map({})
a.time="2018-03-28T8:05:00Z"
a.measurement = 'device1'
a.tags = Map({})
a.tags.user= 'sada'
a.tags.Id = '2'
a.fields=Map({})
a.fields.val1 = 128
a.fields.val2 = 129
client.write_points([a])

True

In [63]:
client.switch_database('mydb')
res=client.query('select * from device1')
res.raw['series']

[{'name': 'device1',
  'columns': ['time', 'Id', 'user', 'val', 'val1', 'val2'],
  'values': [['2018-03-28T08:01:00Z', '1', 'Carol', 127, None, None],
   ['2018-03-28T08:01:00Z', '2', 'sada', None, 128, 129],
   ['2018-03-28T08:02:00Z', '2', 'sada', None, 128, 129],
   ['2018-03-28T08:03:00Z', '2', 'sada', None, 128, 129],
   ['2018-03-28T08:04:00Z', '1', 'Carol', 128, None, None],
   ['2018-03-28T08:04:00Z', '2', 'sada', None, 128, 129],
   ['2018-03-28T08:05:00Z', '2', 'sada', None, 128, 129]]}]

In [68]:
res=client.query("select val1,val2 from device1 where time >= '2018-03-28T08:02:00Z' and time < '2018-03-28T08:05:00Z'")
res.raw['series']

[{'name': 'device1',
  'columns': ['time', 'val1', 'val2'],
  'values': [['2018-03-28T08:02:00Z', 128, 129],
   ['2018-03-28T08:03:00Z', 128, 129],
   ['2018-03-28T08:04:00Z', 128, 129]]}]

In [102]:
%%time

df=LoadDataSet("data/test.csv")
for j,i in df.iterrows():
    a = {}
    a['measurement'] = 'test'
    f={}
    for c in df.columns:
        if ( c.lower().strip() == 'time'):
            a['time'] = int(i[c])
        else:
            f[c.lower()] = i[c] 
    a['fields'] = f
    client.write_points([a])
    if ( j < 5):
        print(j, a)
    #break;
        

0 {'measurement': 'test', 'time': 60000, 'fields': {'a': 94.50227863, 'b': 156.4725017, 'c': 148.5, 'd': 64.51607384, 'e': 25.059642899999997, 'f': 50.30968109, 'g': 47.87570927}}
1 {'measurement': 'test', 'time': 120000, 'fields': {'a': 94.80619537, 'b': 156.7063733, 'c': 148.5, 'd': 64.11511047, 'e': 24.95748489, 'f': 52.31889863, 'g': 47.93354604}}
2 {'measurement': 'test', 'time': 180000, 'fields': {'a': 94.60203514, 'b': 156.69114390000001, 'c': 148.5, 'd': 64.18934813, 'e': 24.69086516, 'f': 53.45025408, 'g': 48.52431744}}
3 {'measurement': 'test', 'time': 240000, 'fields': {'a': 94.67035288, 'b': 156.4964544, 'c': 148.5, 'd': 62.64308376, 'e': 24.72561411, 'f': 53.90358595, 'g': 49.28003926}}
4 {'measurement': 'test', 'time': 300000, 'fields': {'a': 94.77344573, 'b': 156.81235180000002, 'c': 148.5, 'd': 62.3979546, 'e': 24.9269014, 'f': 51.92275418, 'g': 49.67810479}}
CPU times: user 5.09 s, sys: 411 ms, total: 5.5 s
Wall time: 7.55 s


In [110]:
res=client.query("select a from test")
res.raw['series'][0]['values'][0:10]

[['1970-01-01T00:00:00.00006Z', 94.50227863],
 ['1970-01-01T00:00:00.00012Z', 94.80619537],
 ['1970-01-01T00:00:00.00018Z', 94.60203514],
 ['1970-01-01T00:00:00.00024Z', 94.67035288],
 ['1970-01-01T00:00:00.0003Z', 94.77344573],
 ['1970-01-01T00:00:00.00036Z', 94.98822593],
 ['1970-01-01T00:00:00.00042Z', 95.13612635],
 ['1970-01-01T00:00:00.00048Z', 94.94006384],
 ['1970-01-01T00:00:00.00054Z', 95.06376269],
 ['1970-01-01T00:00:00.0006Z', 94.70015668]]

In [121]:
client1 = InfluxDBClient(host='localhost', port=8086, username=None, password=None, database='mydb', ssl=False, verify_ssl=False)
res=client1.query("select a,b,c from test")
res.raw['series'][0]['values'][0:10]

[['1970-01-01T00:00:00.00006Z', 94.50227863, 156.4725017, 148.5],
 ['1970-01-01T00:00:00.00012Z', 94.80619537, 156.7063733, 148.5],
 ['1970-01-01T00:00:00.00018Z', 94.60203514, 156.69114390000001, 148.5],
 ['1970-01-01T00:00:00.00024Z', 94.67035288, 156.4964544, 148.5],
 ['1970-01-01T00:00:00.0003Z', 94.77344573, 156.81235180000002, 148.5],
 ['1970-01-01T00:00:00.00036Z', 94.98822593, 156.862332, 148.5],
 ['1970-01-01T00:00:00.00042Z', 95.13612635, 156.44265359999997, 148.5],
 ['1970-01-01T00:00:00.00048Z', 94.94006384, 156.6491123, 148.5],
 ['1970-01-01T00:00:00.00054Z', 95.06376269, 156.52919690000002, 148.5],
 ['1970-01-01T00:00:00.0006Z', 94.70015668, 156.53588380000002, 148.5]]

In [127]:
res.raw['series']

[{'name': 'test',
  'columns': ['time', 'a', 'b', 'c'],
  'values': [['1970-01-01T00:00:00.00006Z', 94.50227863, 156.4725017, 148.5],
   ['1970-01-01T00:00:00.00012Z', 94.80619537, 156.7063733, 148.5],
   ['1970-01-01T00:00:00.00018Z', 94.60203514, 156.69114390000001, 148.5],
   ['1970-01-01T00:00:00.00024Z', 94.67035288, 156.4964544, 148.5],
   ['1970-01-01T00:00:00.0003Z', 94.77344573, 156.81235180000002, 148.5],
   ['1970-01-01T00:00:00.00036Z', 94.98822593, 156.862332, 148.5],
   ['1970-01-01T00:00:00.00042Z', 95.13612635, 156.44265359999997, 148.5],
   ['1970-01-01T00:00:00.00048Z', 94.94006384, 156.6491123, 148.5],
   ['1970-01-01T00:00:00.00054Z', 95.06376269, 156.52919690000002, 148.5],
   ['1970-01-01T00:00:00.0006Z', 94.70015668, 156.53588380000002, 148.5],
   ['1970-01-01T00:00:00.00066Z', 94.77720753, 156.6049029, 148.5],
   ['1970-01-01T00:00:00.00072Z', 94.27634795, 156.97725119999998, 148.5],
   ['1970-01-01T00:00:00.00078Z', 95.09310619, 156.8059082, 148.5],
   ['1970-0